# Correlation exclusions

The MISP **correlation exclusion** list allows you to document threat values where you do not want correlation to take place. 

You can use this for values that are not relevant for your organisation, for values that provide little additional context or values that are purely informational (and you maybe forgot to disable the correlation flag during ingestion).

## Configure the MISP environment

Import the python libraries and load a **keys** file. This keys file contains the URL and API key to access MISP.

In [3]:
import urllib3
from pymisp import PyMISP, MISPCorrelationExclusion
from keys import misp_url, misp_key, misp_verifycert

if misp_verifycert is False:
    import urllib3
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

misp = PyMISP(misp_url, misp_key, misp_verifycert)

print("I will use the server {}".format(misp_url))

The version of PyMISP recommended by the MISP instance (2.4.160) is newer than the one you're using now (2.4.159). Please upgrade PyMISP.


I will use the server https://misp.demo.cudeso.be


## PyMISP

The correlation exclusions are supported by PyMISP.

### List the correlation exclusions

First get all the existing correlation exclusions.

In [4]:
correlation_exclusions = misp.correlation_exclusions(pythonify = True)
for exclusion in correlation_exclusions:
    print("{};{};{}".format(exclusion.id, exclusion.value, exclusion.comment))

6;corp.sharepoint.com;Internal SharePoint server
7;192.168.8.8;Internal pri DNS server
8;Alert;


### Add a correlation exclusion

Now add a new correlation. A correlation requires at least a value. The comment is optional but recommended for documentation purposes.

In [5]:
new_exclusion = MISPCorrelationExclusion()
new_exclusion.value = "8.8.4.4"
new_exclusion.comment = "Google DNS server"
print(misp.add_correlation_exclusion(new_exclusion))

{'CorrelationExclusion': {'id': '11', 'value': '8.8.4.4', 'from_json': False, 'comment': 'Google DNS server'}}


### Delete a correlation exclusion

Next to adding exclusions, you can also delete an exclusion. To delete a correlation exclusion you need the ID.

In [6]:
exclusion_to_delete = 11
print(misp.delete_correlation_exclusion(exclusion_to_delete))

{'saved': True, 'success': True, 'name': 'CorrelationExclusion deleted.', 'message': 'CorrelationExclusion deleted.', 'url': '/correlation_exclusions/delete/11', 'id': 11}


### Cleanup the correlation exclusions

Adding correlation exclusions is all fine and well if you start with a clean MISP. This is often not the case. You can **clean up existing correlations**. If they match with your exclusions then they are removed from the correlations.

In [7]:
print(misp.clean_correlation_exclusions())

{'saved': True, 'success': True, 'name': 'Correlations cleanup initiated, based on the exclusion rules.', 'message': 'Correlations cleanup initiated, based on the exclusion rules.', 'url': '/correlation_exclusions/clean'}


## Access the correlations without PyMISP

Do not despair if you do not have PyMISP installed. You can manipulate the correlation exclusions also directly via the requests library. First import the library and set the necessary HTTP headers.

In [8]:
import requests

misp_headers = {
            "Authorization": misp_key,
            "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
            "Accept": "application/json",
            "X-Requested-With": "XMLHttpRequest"
        }

In [9]:
url_list_exclusions = "{}/correlation_exclusions/index.json".format(misp_url)
res = requests.get(url_list_exclusions, headers=misp_headers, verify=misp_verifycert)
print(res.text)

[{"CorrelationExclusion":{"id":"6","value":"corp.sharepoint.com","from_json":false,"comment":"Internal SharePoint server"}},{"CorrelationExclusion":{"id":"7","value":"192.168.8.8","from_json":false,"comment":"Internal pri DNS server"}},{"CorrelationExclusion":{"id":"8","value":"Alert","from_json":false,"comment":""}}]


### Add a correlation exclusions with requests

In [10]:
url_add_exclusion = "{}/correlation_exclusions/add".format(misp_url)        
exclusion_to_add = "9.9.9.9"
exclusion_comment = "Another DNS server"
data = "data[CorrelationExclusion][value]={}&data[CorrelationExclusion][comment]={}".format(exclusion_to_add, exclusion_comment) 
res = requests.post(url_add_exclusion, data=data, headers=misp_headers, verify=misp_verifycert)
print(res.text)

{"CorrelationExclusion":{"id":"12","value":"9.9.9.9","from_json":false,"comment":"Another DNS server"}}


### Delete a correlation exclusion with requests

In [11]:
exclusion_to_delete = "12"
url_delete_exclusion = "{}/correlation_exclusions/delete/{}".format(misp_url, exclusion_to_delete)
res = requests.post(url_delete_exclusion, headers=misp_headers, verify=misp_verifycert)
print(res.text)

{"saved":true,"success":true,"name":"CorrelationExclusion deleted.","message":"CorrelationExclusion deleted.","url":"/correlation_exclusions/delete/12","id":12}


## Staying in control of MISP correlations

Also see [Staying in control of MISP correlations](https://www.vanimpe.eu/2021/03/25/staying-in-control-of-misp-correlations/), as well as the new correlation engine support introduced in MISP 2.4.160.